In [20]:
#importing the required modules
import os, json, csv
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk
import numpy as np
from datetime import datetime, timedelta

In [21]:
dataset_path = "plates"  #path containing the Dubai number plates
img_height = 224 
img_width = 224
batch_size = 32  #number of images processing in one training batch
epochs = 15  #training iterations
model_path = "trained_dubai_model.keras"    #poth to save the trained model
class_names_path = "dubai_class_names.json"   #store the class labels

In [22]:
#loading dataset as 80% for training
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,    #20% reserved for testing
    subset="training",
    seed=123,    #consistent date shuffling
    image_size=(img_height, img_width),
    batch_size=batch_size
)

#loading dataset as 20% for testing
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Dubai Plate Classes:", class_names)

Found 348 files belonging to 10 classes.
Using 279 files for training.
Found 348 files belonging to 10 classes.
Using 69 files for validation.
Dubai Plate Classes: ['C5038', 'D77988', 'H79111', 'I9269', 'Q5121', 'R6729', 'S3227', 'S3601', 'U8804', 'X89099']


In [23]:
#performance optimization
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

#augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.2),
])